In [1]:
import numpy as np
import pandas as pd 
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import chi2, SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import joblib

In [2]:
data_path = r'data/news.csv'

In [ ]:
df = pd.read_csv(data_path) 

In [4]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [6]:
df.drop(df.columns[0], axis=1, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
dtypes: object(3)
memory usage: 148.6+ KB


In [8]:
target = "label"
x = df.drop(target, axis=1)
y = df[target]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100, stratify=y)

In [10]:
preprocessor = ColumnTransformer(transformers=[
    ("title_ft", TfidfVectorizer(stop_words="english", ngram_range=(1, 1)), "title"),
    ("text_ft", TfidfVectorizer(stop_words="english", ngram_range=(1, 2), min_df=0.01, max_df=0.95), "text"),
])

In [11]:
cls_decision_tree = Pipeline(steps=[
    ("preprocessor", preprocessor),   
    ("feature_selector", SelectPercentile(chi2, percentile=5)),
    ("model", DecisionTreeClassifier(random_state=42)),
])

params_decision_tree = {
    "model__criterion": ["gini", "entropy","log_loss"],
    "model__max_depth": [10, 20, 30, 50],
    "feature_selector__percentile": [1,5, 10]
}

In [12]:
grid_search_decision_tree = GridSearchCV(estimator=cls_decision_tree, param_grid=params_decision_tree, cv=4, scoring="recall_weighted", verbose=2)
grid_search_decision_tree.fit(x_train, y_train)
y_predicted_decision_tree = grid_search_decision_tree.predict(x_test)

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=10; total time=   4.5s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=10; total time=   5.9s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=10; total time=   5.8s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=10; total time=   5.1s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=20; total time=   5.3s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=20; total time=   6.5s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=20; total time=   7.6s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=20; total time=  10.8s
[CV] END feature_selector__percentile=1, model__criterion=gini, model__max_depth=30; total time=  10.8s
[C

In [13]:
print(classification_report(y_test, y_predicted_decision_tree))

              precision    recall  f1-score   support

        FAKE       0.83      0.86      0.84       633
        REAL       0.86      0.82      0.84       634

    accuracy                           0.84      1267
   macro avg       0.84      0.84      0.84      1267
weighted avg       0.84      0.84      0.84      1267



In [14]:
print(grid_search_decision_tree.best_params_)

{'feature_selector__percentile': 5, 'model__criterion': 'gini', 'model__max_depth': 20}


In [15]:
print(grid_search_decision_tree.best_estimator_)
best_model_decision_tree = grid_search_decision_tree.best_estimator_
joblib.dump(best_model_decision_tree, 'model/decision_tree_model.pkl')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('title_ft',
                                                  TfidfVectorizer(stop_words='english'),
                                                  'title'),
                                                 ('text_ft',
                                                  TfidfVectorizer(max_df=0.95,
                                                                  min_df=0.01,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'text')])),
                ('feature_selector',
                 SelectPercentile(percentile=5,
                                  score_func=<function chi2 at 0x0000023B6ACEACA0>)),
                ('model',
                 Decisio

['model/decision_tree_model.pkl']

In [23]:
cls_svm = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("feature_selector", SelectPercentile(chi2, percentile=5)),
    ("model", SVC(probability=True)) 
])

params_svm = {
    "model__C": [0.1, 1, 10],
    "model__kernel": ['linear', 'poly', 'rbf', 'sigmoid'], 
    "feature_selector__percentile": [1,5, 10]
}

In [24]:
grid_search_svm = GridSearchCV(estimator=cls_svm, param_grid=params_svm, cv=4, scoring="recall_weighted", verbose=2)
grid_search_svm.fit(x_train, y_train)
y_predicted_svm = grid_search_svm.predict(x_test)

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=linear; total time=  14.2s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=linear; total time=  17.4s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=linear; total time=  23.7s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=linear; total time=  23.5s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=poly; total time=  22.4s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=poly; total time=  22.2s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=poly; total time=  21.8s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=poly; total time=  21.4s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=rbf; total time=  20.2s
[CV] END feature_selector__percentile=1, model__C=0.1, model__kernel=rbf; total time=

In [25]:
print(classification_report(y_test, y_predicted_svm))

              precision    recall  f1-score   support

        FAKE       0.91      0.95      0.93       633
        REAL       0.94      0.91      0.93       634

    accuracy                           0.93      1267
   macro avg       0.93      0.93      0.93      1267
weighted avg       0.93      0.93      0.93      1267



In [26]:
print(grid_search_svm.best_params_)

{'feature_selector__percentile': 10, 'model__C': 10, 'model__kernel': 'linear'}


In [27]:
print(grid_search_svm.best_estimator_)
best_model_svm = grid_search_svm.best_estimator_
joblib.dump(best_model_svm, 'model/svm_model.pkl')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('title_ft',
                                                  TfidfVectorizer(stop_words='english'),
                                                  'title'),
                                                 ('text_ft',
                                                  TfidfVectorizer(max_df=0.95,
                                                                  min_df=0.01,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'text')])),
                ('feature_selector',
                 SelectPercentile(score_func=<function chi2 at 0x0000023B6ACEACA0>)),
                ('model', SVC(C=10, kernel='linear', probability=True))])


['model/svm_model.pkl']